In [8]:
import numpy as np
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import glob

# 이미지 파일의 경로를 모두 가져옵니다.
image_paths = glob.glob('/content/drive/MyDrive/Colab Notebooks/10.cj_project/datasets/img_cls/fm_img_hori/train/good/*.jpg')

def create_generator(image_paths, batch_size):
    while True:
        # 배치를 위한 랜덤 인덱스를 생성합니다.
        batch_indices = np.random.randint(0, len(image_paths), batch_size)

        # 배치를 초기화합니다.
        batch_images = []

        for i in batch_indices:
            # 이미지를 로드하고 전처리합니다.
            img = load_img(image_paths[i], target_size=(224, 224))
            img = img_to_array(img) / 255.

            batch_images.append(img)

        batch_images = np.array(batch_images)

        yield batch_images, batch_images  # 입력 이미지와 동일한 이미지를 타겟 데이터로 사용합니다.

# 배치 크기를 설정합니다.
batch_size = 16

# 훈련 데이터와 검증 데이터를 위한 제너레이터를 생성합니다.
train_generator = create_generator(image_paths[int(len(image_paths)*0.2):], batch_size)  # 처음 20%를 제외한 나머지를 훈련 데이터로 사용합니다.
val_generator = create_generator(image_paths[:int(len(image_paths)*0.2)], batch_size)  # 처음 20%를 검증 데이터로 사용합니다.

# Autoencoder 모델을 정의합니다.
input_img = Input(shape=(224, 224, 3))
x = Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (3, 3), activation='relu')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder = Model(input_img, decoded)
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

# EarlyStopping과 ModelCheckpoint 콜백을 정의합니다.
early_stopping = EarlyStopping(monitor='val_loss', patience=10)
model_checkpoint = ModelCheckpoint('b230722_resnet101_anomaly.h5', monitor='val_loss', save_best_only=True)

# Autoencoder를 학습합니다.
autoencoder.fit(train_generator,
                steps_per_epoch=len(image_paths[int(len(image_paths)*0.2):]) // batch_size,
                epochs=100,
                callbacks=[early_stopping, model_checkpoint],
                validation_data=val_generator,
                validation_steps=len(image_paths[:int(len(image_paths)*0.2)]) // batch_size)


Epoch 1/100


InvalidArgumentError: ignored